In [ ]:
%run _prepare.ipynb

In [ ]:
import param
import panel as pn
pn.extension("plotly", "vega")

# `pn.interact` API

In [ ]:
from _helper import *

In [ ]:
params = dict(
    select_country = pn.widgets.MultiSelect(options=countries.country.unique().tolist(), value=["China", "United States"]),
    select_y = pn.widgets.Select(options=countries.select_dtypes(np.number).columns.to_list(), value="co2")
)

def plot(select_country, select_y):
    df = construct_groups(countries, select_country).query("year>1900")
    return pn.Column(
        plot_line(df, select_y),
        plot_area(df, select_y),
        plot_connected_lines(df, select_y)
    )
    
dashboard = pn.interact(plot, **params)

In [ ]:
pn.serve(dashboard)

# Using Panel's Class Structure

In [ ]:
class CountryDashboard(param.Parameterized):
    select_country = pn.widgets.MultiSelect(name="Countries", height=200, options=countries.country.unique().tolist(), value=["China", "United States"])
    select_y = pn.widgets.Select(name="Y-Axis", options=countries.select_dtypes(np.number).columns.to_list(), value="co2")
    
    @param.depends('select_country.value', 'select_y.value')
    def view(self):
        df = construct_groups(countries, self.select_country.value).query("year>1900")
        return pn.Column(
            plot_line(df, self.select_y.value),
            plot_area(df, self.select_y.value),
            plot_connected_lines(df, self.select_y.value)
        )
    
    def render(self):
        return pn.Column(self.select_country, self.select_y, self.view)
        

dashboard = CountryDashboard()

In [ ]:
pn.serve(dashboard.render)